Nombre: Víctor

Apellidos: Toscano Durán

##  Juego 2048

El problema 2048 se basa en un popular juego de fichas que se juega en un tablero de 4×4
. Tiene espacios de estado y acción discretos. El tablero está inicialmente vacío excepto por dos fichas, cada una de las cuales puede tener valor 2 o 4
.
El agente puede mover todas las fichas a la izquierda, abajo, derecha o arriba. La elección de una dirección empuja todas las fichas en esa dirección. Una ficha se detiene cuando choca con una pared o con otra ficha de diferente valor. Una ficha que choca con otra del mismo valor se fusiona con esa ficha, formando una nueva ficha con su valor combinado. En caso de haber más de una opción de fusión, se fusionan primero las fichas que ocupan posiciones más extremas en la dirección del movimiento. Tras el desplazamiento y la fusión, se genera una nueva ficha de valor 2 ó 4en una ficha vacía al azar.

El juego termina cuando ya no podemos desplazar fichas para producir una ficha vacía. Las recompensas se obtienen sólo al fusionar dos fichas, y son iguales al valor de la ficha fusionada.

In [4]:
import random

class Juego2048:
    def __init__(self):
        self.tablero = [[0] * 4 for _ in range(4)]
        self.puntaje = 0
        self.agregar_nueva_ficha()
        self.agregar_nueva_ficha()

    def agregar_nueva_ficha(self):
        casillas_vacias = []
        for i in range(4):
            for j in range(4):
                if self.tablero[i][j] == 0:
                    casillas_vacias.append((i, j))
        
        if casillas_vacias:
            # Asignamos un valor de 2 o 4 a una posición vacía al azar
            i, j = random.choice(casillas_vacias)
            self.tablero[i][j] = random.choices([2, 4], [0.9, 0.1])[0]

    def mover(self, direccion):
        if direccion == "izquierda":
            self.mover_izquierda()
        elif direccion == "derecha":
            self.mover_derecha()
        elif direccion == "arriba":
            self.mover_arriba()
        elif direccion == "abajo":
            self.mover_abajo()

    def mover_izquierda(self):
        nuevo_tablero = [[0] * 4 for _ in range(4)]
        for i in range(4):
            k = 0
            for j in range(4):
                if self.tablero[i][j] != 0:
                    if k > 0 and nuevo_tablero[i][k - 1] == self.tablero[i][j]:
                        nuevo_tablero[i][k - 1] *= 2
                        self.puntaje += nuevo_tablero[i][k - 1]
                    else:
                        nuevo_tablero[i][k] = self.tablero[i][j]
                        k += 1
        self.tablero = nuevo_tablero

    def mover_derecha(self):
        nuevo_tablero = [[0] * 4 for _ in range(4)]
        for i in range(4):
            k = 3
            for j in range(3, -1, -1):
                if self.tablero[i][j] != 0:
                    if k < 3 and nuevo_tablero[i][k + 1] == self.tablero[i][j]:
                        nuevo_tablero[i][k + 1] *= 2
                        self.puntaje += nuevo_tablero[i][k + 1]
                    else:
                        nuevo_tablero[i][k] = self.tablero[i][j]
                        k -= 1
        self.tablero = nuevo_tablero

    def mover_arriba(self):
        nuevo_tablero = [[0] * 4 for _ in range(4)]
        for j in range(4):
            k = 0
            for i in range(4):
                if self.tablero[i][j] != 0:
                    if k > 0 and nuevo_tablero[k - 1][j] == self.tablero[i][j]:
                        nuevo_tablero[k - 1][j] *= 2
                        self.puntaje += nuevo_tablero[k - 1][j]
                    else:
                        nuevo_tablero[k][j] = self.tablero[i][j]
                        k += 1
        self.tablero = nuevo_tablero

    def mover_abajo(self):
        nuevo_tablero = [[0] * 4 for _ in range(4)]
        for j in range(4):
            k = 3
            for i in range(3, -1, -1):
                if self.tablero[i][j] != 0:
                    if k < 3 and nuevo_tablero[k + 1][j] == self.tablero[i][j]:
                        nuevo_tablero[k + 1][j] *= 2
                        self.puntaje += nuevo_tablero[k + 1][j]
                    else:
                        nuevo_tablero[k][j] = self.tablero[i][j]
                        k -= 1
        self.tablero = nuevo_tablero

    def obtener_casillas_vacias(self):
        casillas_vacias = []
        for i in range(4):
            for j in range(4):
                if self.tablero[i][j] == 0:
                    casillas_vacias.append((i, j))
        return casillas_vacias

    def es_fin_del_juego(self):
        for direccion in ["izquierda", "derecha", "arriba", "abajo"]:
            juego_temporal = self.copiar()
            juego_temporal.mover(direccion)
            if juego_temporal.tablero != self.tablero:
                return False
        return True

    def copiar(self):
        copia_juego = Juego2048()
        copia_juego.tablero = [fila[:] for fila in self.tablero]
        copia_juego.puntaje = self.puntaje
        return copia_juego

def calcular_politica_optima(juego):
    def valor_max(juego, alfa, beta, profundidad):
        if juego.es_fin_del_juego() or profundidad == 0:
            return juego.puntaje

        max_utilidad = float('-inf')
        for direccion in ["izquierda", "derecha", "arriba", "abajo"]:
            juego_temporal = juego.copiar()
            juego_temporal.mover(direccion)
            utilidad = valor_min(juego_temporal, alfa, beta, profundidad - 1)
            max_utilidad = max(max_utilidad, utilidad)
            alfa = max(alfa, max_utilidad)
            if beta <= alfa:
                break
        return max_utilidad

    def valor_min(juego, alfa, beta, profundidad):
        if juego.es_fin_del_juego() or profundidad == 0:
            return juego.puntaje

        min_utilidad = float('inf')
        casillas_vacias = juego.obtener_casillas_vacias()
        for casilla in casillas_vacias:
            juego_temporal = juego.copiar()
            juego_temporal.tablero[casilla[0]][casilla[1]] = 2
            utilidad = valor_max(juego_temporal, alfa, beta, profundidad - 1) * 0.9
            min_utilidad = min(min_utilidad, utilidad)
            alfa = min(alfa, min_utilidad)
            if beta <= alfa:
                break

            juego_temporal = juego.copiar()
            juego_temporal.tablero[casilla[0]][casilla[1]] = 4
            utilidad = valor_max(juego_temporal, alfa, beta, profundidad - 1) * 0.1
            min_utilidad = min(min_utilidad, utilidad)
            alfa = min(alfa, min_utilidad)
            if beta <= alfa:
                break
        return min_utilidad

    politica_optima = {}
    for direccion in ["izquierda", "derecha", "arriba", "abajo"]:
        juego_temporal = juego.copiar()
        juego_temporal.mover(direccion)
        utilidad = valor_min(juego_temporal, float('-inf'), float('inf'), profundidad=4)
        politica_optima[direccion] = utilidad

    return politica_optima

# Ejemplo de uso
juego = Juego2048()
print("Tablero inicial:")
for fila in juego.tablero:
    print(fila)

politica_optima = calcular_politica_optima(juego)
print("Política óptima:")
for direccion, utilidad in politica_optima.items():
    print(f"{direccion}: {utilidad}")

Tablero inicial:
[0, 0, 0, 0]
[0, 2, 0, 0]
[0, 0, 0, 0]
[0, 2, 0, 0]
Política óptima:
izquierda: 0.0
derecha: 0.0
arriba: 0.12000000000000002
abajo: 0.12000000000000002


Un ejemplo de como jugar al juego, así como la política óptima en cada movimiento(los valores de utilizar cada acción en cada movimiento del juego). Por ejemplo, seleccionando siempre la acción arriba.

In [6]:
juego = Juego2048()

print("Tablero inicial:")
for fila in juego.tablero:
    print(fila)
print("Puntuación inicial:", juego.puntaje)

while not juego.es_fin_del_juego():
    print("\n--- Nuevo movimiento ---")
    print("Tablero:")
    for fila in juego.tablero:
        print(fila)

    politica_optima = calcular_politica_optima(juego)
    print("Política óptima:")
    for direccion, utilidad in politica_optima.items():
        print(f"{direccion}: {utilidad}")

    movimientos_validos = ["izquierda", "derecha", "arriba", "abajo"]
    movimiento = input("Elige una dirección de movimiento (izquierda/derecha/arriba/abajo): ")
    while movimiento not in movimientos_validos:
        print("Movimiento inválido. Inténtalo de nuevo.")
        movimiento = input("Elige una dirección de movimiento (izquierda/derecha/arriba/abajo): ")

    juego.mover(movimiento)
    juego.agregar_nueva_ficha()

print("\n--- Juego terminado ---")
print("Tablero final:")
for fila in juego.tablero:
    print(fila)
print("Puntuación final:", juego.puntaje)

Tablero inicial:
[0, 0, 0, 0]
[2, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 2]
Puntuación inicial: 0

--- Nuevo movimiento ---
Tablero:
[0, 0, 0, 0]
[2, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 2]
Política óptima:
izquierda: 0.0
derecha: 0.0
arriba: 0.0
abajo: 0.0
Elige una dirección de movimiento (izquierda/derecha/arriba/abajo): arriba

--- Nuevo movimiento ---
Tablero:
[2, 0, 2, 2]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
Política óptima:
izquierda: 0.04000000000000001
derecha: 0.04000000000000001
arriba: 0.12000000000000002
abajo: 0.12000000000000002
Elige una dirección de movimiento (izquierda/derecha/arriba/abajo): arriba

--- Nuevo movimiento ---
Tablero:
[2, 0, 2, 2]
[0, 0, 2, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
Política óptima:
izquierda: 0.04000000000000001
derecha: 0.04000000000000001
arriba: 0.12000000000000002
abajo: 0.12000000000000002
Elige una dirección de movimiento (izquierda/derecha/arriba/abajo): arriba

--- Nuevo movimiento ---
Tablero:
[2, 0, 4, 2]
[0, 0, 0, 0]
[0, 0, 0, 0]
[2, 0, 0, 0]
Polí

KeyboardInterrupt: Interrupted by user

### Explicación código

Se ha creado una clase llamada Juego2048 para implementar el problema(juego) 2048. Cada método de la clase tiene la siguiente funcionalidad:

- init(self): El método de inicialización de la clase. Establece el estado inicial del juego, creando un tablero vacío de 4x4, estableciendo el puntaje en 0 y agregando dos nuevas fichas al azar al tablero.

- agregar_nueva_ficha(self): Agrega una nueva ficha (con valor 2 o 4) en una casilla vacía al azar en el tablero.

- mover(self, direccion): Mueve todas las fichas del juego en la dirección seleccionada (izquierda, derecha, arriba o abajo). Llama a alguno de los siguientes métodos en función de la dirección dada.

- mover_izquierda(self): Mueve todas las fichas del juego hacia la izquierda, fusionando fichas del mismo valor cuando se encuentran. Actualiza el tablero y el puntaje en consecuencia.

- mover_derecha(self): Similar a mover_izquierda, pero mueve las fichas hacia la derecha.

- mover_arriba(self): Mueve todas las fichas del juego hacia arriba intercambiando filas y columnas, fusionando fichas del mismo valor cuando se encuentran. Actualiza el tablero y el puntaje en consecuencia.

- mover_abajo(self): Similar a mover_arriba, pero mueve las fichas hacia abajo.

- obtener_casillas_vacias(self): Devuelve una lista de las coordenadas de las casillas vacías en el tablero.

- es_fin_del_juego(self): Verifica si no se pueden realizar más movimientos en el juego. Comprueba si al mover en cualquier dirección el tablero sigue siendo el mismo, lo que indicaría que no hay más movimientos posibles.

- copiar(self): Crea una copia del juego, incluyendo el tablero y el puntaje.

La siguiente función calcula la política optima en cada movimiento del juego:

calcular_politica_optima(juego): Calcula la política óptima del juego utilizando el algoritmo Minimax. Realiza una búsqueda en el árbol de posibles movimientos hasta una cierta profundidad y asigna una utilidad a cada movimiento.